# Build a Web Research Agent with Tavily API 🌐 🟠

Welcome! In this tutorial, you'll learn how to build a web research agent using [Tavily API](https://docs.tavily.com/documentation/api-reference/introduction) that can search, extract, crawl, and reason over live web data.

These skills are essential for anyone building AI agents or applications that need up-to-date, relevant information from the internet. By learning how to programmatically access and process real-time web data, you'll be able to bridge the gap between static language models and the dynamic world they operate in, making your agents smarter, more accurate, and context-aware.

The AWS Strands Agent Framework enables rapid development of AI agents with minimal code. Many research agent implementations require extensive development efforts and rely on deterministic logic with constrained inputs and outputs. Alternatively, Strands facilitates building highly dynamic agents through natural language. Strands agents leverage prompt engineering to dynamically generate varied output types and accept diverse natural language inputs seamlessly.

The core philosophy of Strands shifts complexity from hard-coded logic directly into the weights of the LLM, granting the model significant autonomy to determine agent behavior. This design approach ensures agents remain highly flexible and scalable, easily benefiting from advancements in new model releases. By simply integrating updated LLMs, developers can immediately unlock significant performance improvements without needing to modify any existing agent logic.

By the end of this lesson, you'll know how to:
- Connect agents to the web for up-to-date research
- Orchestrate the web tools dynamically with the Strands agent framework
- Build dynamic research agents capable of performing a range of tasks, including deep research, report writing, direct question answering, list building, etc.



---

## Getting Started

Follow these steps to set up:

1. **Sign up** for Tavily at [app.tavily.com](https://app.tavily.com/home/) to get your API key.

   *Refer to the screenshots linked below for step-by-step guidance:*

<div style="text-align:center">
    <img src="assets/sign-up.png" width="65%" />
</div>

<div style="text-align:center">
    <img src="assets/api-key.png" width="65%" />
</div>

2. **Copy your API key** from your Tavily account dashboard.

3. **Paste your API key** into the cell below and execute the cell.

In [ ]:
# To export your API key into a .env file, run the following cell (PLEASE REPLACE WITH YOUR API KEY):
!echo "TAVILY_API_KEY=<your-tavily-api-key>" >> .env

Install and import necessary dependencies.

In [ ]:
%pip install strands-agents tavily-python --quiet

### Setting Up Your Tavily API Client

The code below will instantiate the Tavily client with your API key.

In [2]:
import os
import getpass
from dotenv import load_dotenv
from tavily import TavilyClient

# Load environment variables from .env file
load_dotenv()

# Prompt the user to securely input the API key if not already set in the environment
if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY:\n")

# Initialize the Tavily API client using the loaded or provided API key
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

> **ℹ️ Strands Agent Architecture**
>
> This research agent is composed of three primary components:
>
> 1. **Language Model(LLM):** Acts as the agent's "brain," responsible for understanding queries and generating responses.
> 2. **Tools:** Includes `web search`, `web extract`, and `web crawl` functionalities, enabling the agent to interact with and gather information from the internet. Also includes a `research formatting` tool to allow the agent to dynamically alter the research output format based on the user's intent.
> 3. **System Prompt:** Guides the agent's behavior, outlining how and when to use each tool to achieve its research objectives.

Using these 3 major components, we will create this architecture in this notebook:

<div style="text-align:center">
    <img src="assets/architecture.png" width="65%" />
</div>


## 1. Language Model

We'll use the Strands SDK to set up a language model for our agent via AWS Bedrock. In this example, we're choosing Anthropic's Claude 4 Sonnet model, but you can substitute any [Bedrock-supported model](https://docs.aws.amazon.com/bedrock/latest/userguide/models-supported.html) as needed. Before you can use a foundation model in Amazon Bedrock, you must request access to it. For instructions, see [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) in the Amazon Bedrock User Guide.

In [33]:
from strands.models import BedrockModel

bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1",
)

# 2. Tool Definitions 
Let's define the following modular tools with the Tavily-LangChain integration:
1. **Search** the web for relevant information

2. **Extract** the full page content from a webpage

2. **Crawl** entire websites and scrape their content

3. **Format Responses** dynamically using an LLM


### Define the Tavily Search Tool 🔍

We'll wrap the Tavily search endpoint in the Strands `@tool` decorator. Tools are passed to agents during initialization or at runtime, making them available for use throughout the agent's lifecycle. We implement a `format_search_results_for_agent` helper function which parses Tavily search results into a clear, structured format that's easy for the LLM to process. 

The agent will have the ability to set the query, time range, and include domains parameters. Feel free to experiment with different Tavily API parameter configurations to see Tavily in action. You can adjust parameters such as the number of results, time range, and domain filters to tailor your search. For more information, read the [search API reference](https://docs.tavily.com/documentation/api-reference/endpoint/search) and [best practices guide](https://docs.tavily.com/documentation/best-practices/best-practices-search). 


In [14]:
from strands import Agent, tool


def format_search_results_for_agent(tavily_result):
    """
    Format Tavily search results into a well-structured string for language models.

    Args:
        tavily_result (Dict): A Tavily search result dictionary

    Returns:
        str: A formatted string with search results organized for easy consumption by LLMs
    """
    if (
        not tavily_result
        or "results" not in tavily_result
        or not tavily_result["results"]
    ):
        return "No search results found."

    formatted_results = []

    for i, doc in enumerate(tavily_result["results"], 1):
        # Extract metadata
        title = doc.get("title", "No title")
        url = doc.get("url", "No URL")

        # Create a formatted entry
        formatted_doc = f"\nRESULT {i}:\n"
        formatted_doc += f"Title: {title}\n"
        formatted_doc += f"URL: {url}\n"

        raw_content = doc.get("raw_content")

        # Prefer raw_content if it's available and not just whitespace
        if raw_content and raw_content.strip():
            formatted_doc += f"Raw Content: {raw_content.strip()}\n"
        else:
            # Fallback to content if raw_content is not suitable or not available
            content = doc.get("content", "").strip()
            formatted_doc += f"Content: {content}\n"

        formatted_results.append(formatted_doc)

    # Join all formatted results with a separator
    return "\n" + "\n".join(formatted_results)


@tool
def web_search(
    query: str, time_range: str | None = None, include_domains: str | None = None
) -> str:
    """Perform a web search. Returns the search results as a string, with the title, url, and content of each result ranked by relevance.

    Args:
        query (str): The search query to be sent for the web search.
        time_range (str | None, optional): Limits results to content published within a specific timeframe.
            Valid values: 'd' (day - 24h), 'w' (week - 7d), 'm' (month - 30d), 'y' (year - 365d).
            Defaults to None.
        include_domains (list[str] | None, optional): A list of domains to restrict search results to.
            Only results from these domains will be returned. Defaults to None.

    Returns:
        formatted_results (str): The web search results
    """
    client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
    formatted_results = format_search_results_for_agent(
        client.search(
            query=query,  # The search query to execute with Tavily.
            max_results=10,
            time_range=time_range,
            include_domains=include_domains,  # list of domains to specifically include in the search results.
        )
    )
    return formatted_results

### Define the Tavily Extract Tool 📄

We'll wrap the Tavily extract endpoint in the Strands `@tool` decorator to retrieve the complete content (i.e., `raw_content`) of web pages. For efficiency, the extract endpoint can process up to 20 URLs at once in a single call

For more information, read the [extract API reference](https://docs.tavily.com/documentation/api-reference/endpoint/extract) and [best practices guide](https://docs.tavily.com/documentation/best-practices/best-practices-extract). 


In [51]:
def format_extract_results_for_agent(tavily_result):
    """
    Format Tavily extract results into a well-structured string for language models.

    Args:
        tavily_result (Dict): A Tavily extract result dictionary

    Returns:
        str: A formatted string with extract results organized for easy consumption by LLMs
    """
    if not tavily_result or "results" not in tavily_result:
        return "No extract results found."

    formatted_results = []

    # Process successful results
    results = tavily_result.get("results", [])
    for i, doc in enumerate(results, 1):
        url = doc.get("url", "No URL")
        raw_content = doc.get("raw_content", "")
        images = doc.get("images", [])

        formatted_doc = f"\nEXTRACT RESULT {i}:\n"
        formatted_doc += f"URL: {url}\n"

        if raw_content:
            # Truncate very long content for readability
            if len(raw_content) > 5000:
                formatted_doc += f"Content: {raw_content[:5000]}...\n"
            else:
                formatted_doc += f"Content: {raw_content}\n"
        else:
            formatted_doc += "Content: No content extracted\n"

        if images:
            formatted_doc += f"Images found: {len(images)} images\n"
            for j, image_url in enumerate(images[:3], 1):  # Show up to 3 images
                formatted_doc += f"  Image {j}: {image_url}\n"
            if len(images) > 3:
                formatted_doc += f"  ... and {len(images) - 3} more images\n"

        formatted_results.append(formatted_doc)

    # Process failed results if any
    failed_results = tavily_result.get("failed_results", [])
    if failed_results:
        formatted_results.append("\nFAILED EXTRACTIONS:\n")
        for i, failure in enumerate(failed_results, 1):
            url = failure.get("url", "Unknown URL")
            error = failure.get("error", "Unknown error")
            formatted_results.append(f"Failed {i}: {url} - {error}\n")

    # Add response time info
    response_time = tavily_result.get("response_time", 0)
    formatted_results.append(f"\nResponse time: {response_time} seconds")

    return "\n" + "".join(formatted_results)


@tool
def web_extract(
    urls: str | list[str], include_images: bool = False, extract_depth: str = "basic"
) -> str:
    """Extract content from one or more web pages using Tavily's extract API.

    Args:
        urls (str | list[str]): A single URL string or a list of URLs to extract content from.
        include_images (bool, optional): Whether to also extract image URLs from the pages.
                                       Defaults to False.
        extract_depth (str, optional): The depth of extraction. 'basic' provides standard
                                     content extraction, 'advanced' provides more detailed
                                     extraction. Defaults to "basic".

    Returns:
        str: A formatted string containing the extracted content from each URL, including
             the full raw content, any images found (if requested), and information about
             any URLs that failed to be processed.
    """
    try:
        # Ensure urls is always a list for the API call
        if isinstance(urls, str):
            urls_list = [urls]
        else:
            urls_list = urls

        # Clean and validate URLs
        cleaned_urls = []
        for url in urls_list:
            if url.strip().startswith("{") and '"url":' in url:
                import re

                m = re.search(r'"url"\s*:\s*"([^"]+)"', url)
                if m:
                    url = m.group(1)

            if not url.startswith(("http://", "https://")):
                url = "https://" + url

            cleaned_urls.append(url)

        # Call Tavily extract API
        api_response = tavily_client.extract(
            urls=cleaned_urls,  # List of URLs to extract content from
            include_images=include_images,  # Whether to include image extraction
            extract_depth=extract_depth,  # Depth of extraction (basic or advanced)
        )

        # Format the results for the agent
        formatted_results = format_extract_results_for_agent(api_response)
        return formatted_results

    except Exception as e:
        return f"Error during extraction: {e}\nURLs attempted: {urls}\nFailed to extract content."

### Define the Tavily Crawl Tool 🕸️ 

Now let’s use Tavily to crawl a webpage and extract all its links. Web crawling is the process of automatically navigating through websites by following hyperlinks to discover numerous web pages and URLs (think of it like falling down a Wikipedia rabbit hole 🐇—clicking from page to page, diving deeper into interconnected topics). For autonomous web agents, this capability is essential for accessing deep web data which might be difficult to retrieve via search. 


We'll wrap the Tavily crawl endpoint in the Strands `@tool` decorator, similar to the search tool. We implement a `format_crawl_results_for_agent` helper function which parses Tavily search results into a clear, structured format that's easy for the LLM to process. 

The agent will have the ability to set the crawled url and the crawl instruction. You can adjust parameters such as the crawl depth, limit, and domain filters to tailor your crawl. For more information, read the crawl [API reference](https://docs.tavily.com/documentation/api-reference/endpoint/crawl) and [best practices guide](https://docs.tavily.com/documentation/best-practices/best-practices-crawl).

In [5]:
def format_crawl_results_for_agent(tavily_result):
    """
    Format Tavily crawl results into a well-structured string for language models.

    Args:
        tavily_result (List[Dict]): A list of Tavily crawl result dictionaries

    Returns:
        formatted_results (str): The formatted crawl results
    """
    if not tavily_result:
        return "No crawl results found."

    formatted_results = []

    for i, doc in enumerate(tavily_result, 1):
        # Extract metadata
        url = doc.get("url", "No URL")
        raw_content = doc.get("raw_content", "")

        # Create a formatted entry
        formatted_doc = f"\nRESULT {i}:\n"
        formatted_doc += f"URL: {url}\n"

        if raw_content:
            # Extract a title from the first line if available
            title_line = raw_content.split("\n")[0] if raw_content else "No title"
            formatted_doc += f"Title: {title_line}\n"
            formatted_doc += (
                f"Content: {raw_content[:4000]}...\n"
                if len(raw_content) > 4000
                else f"Content: {raw_content}\n"
            )

        formatted_results.append(formatted_doc)

    # Join all formatted results with a separator
    return "\n" + "-" * 40 + "\n".join(formatted_results)


@tool
def web_crawl(url: str, instructions: str | None = None) -> str:
    """
    Crawls a given URL, processes the results, and formats them into a string.

    Args:
        url (str): The URL of the website to crawl.

        instructions (str | None, optional): Specific instructions to guide the
                                             Tavily crawler, such as focusing on
                                             certain types of content or avoiding
                                             others. Defaults to None.

    Returns:
        str: A formatted string containing the crawl results. Each result includes
             the URL and a snippet of the page content.
             If an error occurs during the crawl process (e.g., network issue,
             API error), a string detailing the error and the attempted URL is
             returned.

    """
    max_depth = 2
    limit = 20

    if url.strip().startswith("{") and '"url":' in url:
        import re

        m = re.search(r'"url"\s*:\s*"([^"]+)"', url)
        if m:
            url = m.group(1)

    if not url.startswith(("http://", "https://")):
        url = "https://" + url

    try:
        # Crawls the web using Tavily API
        api_response = tavily_client.crawl(
            url=url,  # The URL to crawl
            max_depth=max_depth,  # Defines how far from the base URL the crawler can explore
            limit=limit,  # Limits the number of results returned
            instructions=instructions,  # Optional instructions for the crawler
        )

        tavily_results = (
            api_response.get("results")
            if isinstance(api_response, dict)
            else api_response
        )

        formatted = format_crawl_results_for_agent(tavily_results)
        return formatted
    except Exception as e:
        return f"Error: {e}\n" f"URL attempted: {url}\n" "Failed to crawl the website."

### 📝 Research Formatter Tool

The `format_research_response` tool uses a specialized agent to transform raw research content into clear, well-structured, and properly cited responses. It ensures every factual claim is supported by inline citations and provides a complete "Sources" section with URLs. The tool automatically selects the most appropriate format—such as direct answer, blog, academic report, executive summary, bullet points, or comparison—based on the user's question and the research content.

Use this tool as the final step, after completing all research, to transform your findings into a clear, well-structured, and audience-appropriate response before delivering it to the user.


In [19]:
# Define specialized system prompt for research response formatting
RESEARCH_FORMATTER_PROMPT = """
You are a specialized Research Response Formatter Agent. Your role is to transform research content into well-structured, properly cited, and reader-friendly formats.

Core formatting requirements (ALWAYS apply):
1. Include inline citations using [n] notation for EVERY factual claim
2. Provide a complete "Sources" section at the end with numbered references an urls
3. Write concisely - no repetition or filler words
4. Ensure information density - every sentence should add value
5. Maintain professional, objective tone
6. Format your response in markdown

Based on the semantics of the user's original research question, format your response in one of the following styles:
- **Direct Answer**: Concise, focused response that directly addresses the question
- **Blog Style**: Engaging introduction, subheadings, conversational tone, conclusion
- **Academic Report**: Abstract, methodology, findings, analysis, conclusions, references
- **Executive Summary**: Key findings upfront, bullet points, actionable insights
- **Bullet Points**: Structured lists with clear hierarchy and supporting details
- **Comparison**: Side-by-side analysis with clear criteria and conclusions

When format is not specified, analyze the research content and user query to determine:
- Complexity level (simple vs. comprehensive)
- Audience (general public vs. technical)
- Purpose (informational vs. decision-making)
- Content type (factual summary vs. analytical comparison)

Your response below should be polished, containing only the information that is relevant to the user's query and NOTHING ELSE.

Your final research response:
"""


@tool
def format_research_response(
    research_content: str, format_style: str = None, user_query: str = None
) -> str:
    """Format research content into a well-structured, properly cited response.

    This tool uses a specialized Research Formatter Agent to transform raw research
    into polished, reader-friendly content with proper citations and optimal structure.

    Args:
        research_content (str): The raw research content to be formatted
        format_style (str, optional): Desired format style (e.g., "blog", "report",
                                    "executive summary", "bullet points", "direct answer")
        user_query (str, optional): Original user question to help determine appropriate format

    Returns:
        str: Professionally formatted research response with proper citations,
             clear structure, and appropriate style for the intended audience
    """
    try:
        bedrock_model = BedrockModel(
            model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
            region_name="us-east-1",
        )
        # Strands Agents SDK makes it easy to create a specialized agent
        formatter_agent = Agent(
            model=bedrock_model,
            system_prompt=RESEARCH_FORMATTER_PROMPT,
        )

        # Prepare the input for the formatter
        format_input = f"Research Content:\n{research_content}\n\n"

        if format_style:
            format_input += f"Requested Format Style: {format_style}\n\n"

        if user_query:
            format_input += f"Original User Query: {user_query}\n\n"

        format_input += "Please format this research content according to the guidelines and appropriate style."

        # Call the agent and return its response
        response = formatter_agent(format_input)
        return str(response)
    except Exception as e:
        return f"Error in research formatting: {str(e)}"

# 3. Agent System Prompt

The Strands SDK enables the agent to reason about which actions to take, use the available tools in sequence, and iterate as needed until it completes its research task. The system prompt is especially important—it instructs the agent on best practices for using the tools together, ensuring that the agent's responses are thorough, accurate, and well-sourced.

You are encouraged to experiment with the system prompt or try different language models to change the agent's style, personality, or optimize its performance for specific use cases.

In [ ]:
import datetime

today = datetime.datetime.today().strftime("%A, %B %d, %Y")

SYSTEM_PROMPT = f"""
You are an expert research assistant specializing in deep, comprehensive information gathering and analysis.
You are equipped with advanced web tools: Web Search, Web Extract, and Web Crawl.
Your mission is to conduct comprehensive, accurate, and up-to-date research, grounding your findings in credible web sources.

**Today's Date:** {today}

Your TOOLS include:

1. WEB SEARCH
- Conduct thorough web searches using the web_search tool.
- You will enter a search query and the web_search tool will return 10 results ranked by semantic relevance.
- Your search results will include the title, url, and content of 10 results ranked by semantic relevance.

2. WEB EXTRACT
- Conduct web extraction with the web_extract tool.
- You will enter a url and the web_extract tool will extract the content of the page.
- Your extract results will include the url and content of the page.
- This tool is great for finding all the information that is linked from a single page.

3. WEB CRAWL
- Conduct deep web crawls with the web_crawl tool.
- You will enter a url and the web_crawl tool will find all the nested links.
- Your crawl results will include the url and content of the pages that were discovered.
- This tool is great for finding all the information that is linked from a single page.

3. FORMATTING RESEARCH RESPONSE
- You will use the format_research_response tool to format your research response.
- This tool will create a well-structured response that is easy to read and understand.
- The response will clearly address the user's query, the research results.
- The response will be in markdown format.

RULES:
- You must start the research process by creating a plan. Think step by step about what you need to do to answer the research question.
- You can iterate on your research plan and research response multiple times, using combinations of the tools available to you until you are satisfied with the results.
- You must use the format_research_response tool at the end of your research process.

"""

Now let's combine the search and crawl tools into a single agent, as shown in the diagram below.

<div style="text-align:center">
    <img src="assets/agent.svg" width="65%" />
</div>


In [62]:
web_agent = Agent(
    model=bedrock_model,
    system_prompt=SYSTEM_PROMPT,
    tools=[
        web_search,
        web_extract,
        web_crawl,
        format_research_response,
    ],
)

Let's test the agent.

In [65]:
research_prompt = "find the integration partners of the tavily api"

In [ ]:
research = web_agent(research_prompt)

Now lets view the final research output.

In [ ]:
from IPython.display import display, Markdown

# Find the specific tool result for format_research_response
for msg in web_agent.messages:
    if msg.get("role") == "user":
        for content in msg.get("content", []):
            tool_result = content.get("toolResult", {})
            if tool_result.get("status") == "success":
                # Check if this corresponds to format_research_response
                # Look for the toolUseId that matches format_research_response
                tool_use_id = tool_result.get("toolUseId", "")

                # Find the matching tool use in assistant messages
                for assistant_msg in web_agent.messages:
                    if assistant_msg.get("role") == "assistant":
                        for assistant_content in assistant_msg.get("content", []):
                            tool_use = assistant_content.get("toolUse", {})
                            if (
                                tool_use.get("toolUseId") == tool_use_id
                                and tool_use.get("name") == "format_research_response"
                            ):

                                formatted_content = tool_result.get("content", [{}])[
                                    0
                                ].get("text", "")
                                display(Markdown(formatted_content))
                                break

Let's view the tool execution order.

In [ ]:
tools_used = []

print("🚀 Tool Execution Flow")
print("─" * 50)

for i, msg in enumerate(web_agent.messages):
    if msg.get("role") == "assistant" and msg.get("content"):
        content = msg.get("content", [])

        for item in content:
            if isinstance(item, dict) and "toolUse" in item:
                tool_use = item["toolUse"]
                tool_name = tool_use.get("name", "unknown")
                tool_input = tool_use.get("input", {})
                tools_used.append(tool_name)

                # Choose emoji based on tool type
                if "crawl" in tool_name:
                    emoji = "🕷️"
                elif "search" in tool_name:
                    emoji = "🔍"
                elif "format" in tool_name:
                    emoji = "📝"
                elif "extract" in tool_name:
                    emoji = "📄"
                else:
                    emoji = "⚡"

                print(f"{len(tools_used):2d}. {emoji} {tool_name}")

                # Format input nicely
                if isinstance(tool_input, dict):
                    for key, value in tool_input.items():
                        # Truncate long values for readability
                        if isinstance(value, str) and len(value) > 80:
                            value = value[:77] + "..."
                        print(f"    💭 {key}: {value}")
                else:
                    print(f"    💭 input: {tool_input}")
                print()  # Add blank line for readability

print(f"🎯 Completed {len(tools_used)} tool invocations!")

We can view the agent sub steps for monitoring and observability.

In [ ]:
web_agent.messages

In [ ]:
research.metrics